In [235]:
!pip install yfinance

In [236]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def mean_portfolio(weights, *dataopt):
    """Calculate portfolio return"""
    mean_returns = dataopt[0]  # This is the array of expected returns for each asset
    portfolio_return = mean_returns.dot(weights)
    return portfolio_return

def std_portfolio(weights, *dataopt):
    """Calculate portfolio volatility (standard deviation)"""
    covariance_returns = dataopt[1]  # This is the covariance matrix of asset returns
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_returns, weights)))
    return portfolio_volatility

def sharpe_ratio(weights, *dataopt):
    """Calculate Sharpe Ratio (negated for maximization)"""
    mean_returns = dataopt[0]  # Expected annual returns (array)
    cov_matrix = dataopt[1]    # Covariance matrix of returns

    # Calculate portfolio return
    portfolio_return = np.dot(weights, mean_returns)

    # Calculate portfolio volatility (standard deviation)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

    # Compute Sharpe Ratio (negated for maximization)
    sharpe = portfolio_return / portfolio_volatility
    return sharpe  # Negate because `scipy.optimize.minimize` minimizes functions

def portfolio_optimization(monthly_returns, stock):
    """Optimize portfolio using Sharpe Ratio maximization"""
    annual_trading_months = 12
    observed_trading_months = monthly_returns.shape[0]
    
    # Annualized returns
    annualized_returns = (1 + monthly_returns).prod() ** (annual_trading_months / observed_trading_months) - 1
    print("Annualized Stock Return =", annualized_returns)
    # Annualized covariance matrix
    cov_annualized_returns = monthly_returns.cov() * annual_trading_months
    
    # Prepare optimization inputs
    dataopt = (annualized_returns, cov_annualized_returns)
    num_assets = len(annualized_returns)
    
    # Initial equal weights
    initial_weights = np.ones(num_assets) / num_assets
    
    # Bounds (weights between 0 and 1 for long-only portfolios)
    bounds = [(0, 1) for _ in range(num_assets)]
    
    # Constraint: Sum of weights must be 1
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}
    
    # Perform optimization
    result = minimize(
        fun=lambda x, *dataopt: -sharpe_ratio(x, *dataopt),  # Negate return for maximization
        x0=initial_weights,
        args=dataopt,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    optimal_weights_sr = dict(zip(stock, result.x))
    print("Optimal Portfolio Allocation max SR")
    for asset, weight in optimal_weights_sr.items():
        print(f"{asset}: {weight:.2%}")  # Display as percentage with 2 decimal places
    
    # Portfolio metrics
    portfolio_return = mean_portfolio(result.x, *dataopt)
    portfolio_volatility = std_portfolio(result.x, *dataopt)
    sharpe = portfolio_return / portfolio_volatility
    
    print(f"\nExpected Annual Return: {portfolio_return:.2%}")
    print(f"Annual Volatility: {portfolio_volatility:.2%}")
    print(f"Sharpe Ratio: {sharpe:.2f}")

    return optimal_weights  # Return optimal allocation

    
    # to use just write (optimal_weights = portfolio_optimization(sample_data))
    # sample_data is monthly return data


In [237]:
import pandas as pd

# Read clustered data
clustered_df = pd.read_csv(r"Output/models/clustered_df.csv")

# Remove the first column
clustered_df = clustered_df.iloc[:, 1:]

print(clustered_df) 


            Date   Stock  PredictedReturn  PredictedVolatility  Cluster
0     2024-01-31       A        -0.744057             0.102797        0
1     2024-01-31  AAON.O        -0.039090             0.101948        0
2     2024-01-31     AAP         0.630248             0.137507        0
3     2024-01-31  AAPL.O         6.281893             0.080092        3
4     2024-01-31  ABEO.O        -4.939540             0.238952        2
...          ...     ...              ...                  ...      ...
9510  2024-10-31     ZBH        -1.884281             0.076510        2
9511  2024-10-31    ZD.O        -0.798077             0.094849        2
9512  2024-10-31  ZEUS.O        -0.347221             0.156842        2
9513  2024-10-31  ZION.O        -2.849456             0.109860        2
9514  2024-10-31  ZYXI.O         0.427867             0.142379        0

[9515 rows x 5 columns]


In [238]:
# prompt: now filter data only for Date = ['2024-10-31']

import numpy as np
import pandas as pd


# Convert 'Date' column to datetime objects
end_clustered_df= clustered_df.copy()
end_clustered_df['Date'] = pd.to_datetime(end_clustered_df['Date'])

# Filter data for '2024-10-31'
filtered_df = clustered_df[clustered_df['Date'] == '2024-10-31']


# Create dataframes for each cluster
df_cluster0 = filtered_df[filtered_df['Cluster'] == 0]
df_cluster1 = filtered_df[filtered_df['Cluster'] == 1]
df_cluster2 = filtered_df[filtered_df['Cluster'] == 2]
df_cluster3 = filtered_df[filtered_df['Cluster'] == 3]

# Print updated dataframes (optional)
print("df_cluster0:\n", df_cluster0)
print("\ndf_cluster1:\n", df_cluster1.head())
print("\ndf_cluster2:\n", df_cluster2.head())
print("\ndf_cluster3:\n", df_cluster3.head())

df_cluster0:
             Date   Stock  PredictedReturn  PredictedVolatility  Cluster
8558  2024-10-31     AAP        -0.022118         2.359802e-01        0
8559  2024-10-31  AAPL.O         6.741405         7.493543e-02        0
8561  2024-10-31     ABT         0.645390         3.400000e-07        0
8562  2024-10-31  ACAD.O         1.150560         3.220000e-14        0
8563  2024-10-31  ACCS.K        -0.203058         1.690312e-01        0
...          ...     ...              ...                  ...      ...
9504  2024-10-31       X         0.089402         1.272337e-01        0
9505  2024-10-31     XOM         0.300670         5.043796e-02        0
9507  2024-10-31  XRAY.O         1.176372         8.663112e-02        0
9509  2024-10-31  YORW.O         0.344406         5.938221e-02        0
9514  2024-10-31  ZYXI.O         0.427867         1.423785e-01        0

[469 rows x 5 columns]

df_cluster1:
             Date   Stock  PredictedReturn  PredictedVolatility  Cluster
8557  2024-

In [239]:
import pandas as pd

# Read the clustered data CSV file
merged_df = pd.read_csv(r"Output/models/merged_data.csv")

# Filter for 2024-10-31 data only
oct_2024_data = merged_df[merged_df['Period'] == '2024-10-31']

# Select only 'RIC' and 'Company Market Cap' columns and rename RIC to Stock
mktfiltered_data = oct_2024_data[['RIC', 'Company Market Cap']].rename(columns={'RIC': 'Stock'})

# Display the result
print(mktfiltered_data)


         Stock  Company Market Cap
3       AAPL.O        3.210436e+12
220     NVDA.O        2.968748e+12
437     MSFT.O        2.888548e+12
654     AMZN.O        2.097821e+12
871        LLY        7.713173e+11
...        ...                 ...
210493   AWH.O        2.333416e+06
210710  SUNE.O        2.306633e+06
210927  TCRT.O        2.289950e+06
211144  WINT.O        1.435916e+06
211361  DGLY.O        1.375079e+06

[975 rows x 2 columns]


In [240]:
def add_market_cap_to_cluster(cluster_df, market_cap_df):
    """Merge cluster_df with market_cap_df to add 'Company Market Cap'."""
    return cluster_df.merge(
        market_cap_df,
        on='Stock',
        how='left'  # Keep all stocks even if market cap is missing
    )

# Apply to all clusters (replace df_cluster0, df_cluster1, etc. with your actual DataFrames)
df_cluster0_with_cap = add_market_cap_to_cluster(df_cluster0, mktfiltered_data)
df_cluster1_with_cap = add_market_cap_to_cluster(df_cluster1, mktfiltered_data)
df_cluster2_with_cap = add_market_cap_to_cluster(df_cluster2, mktfiltered_data)
df_cluster3_with_cap = add_market_cap_to_cluster(df_cluster3, mktfiltered_data)

# --- Step 3: Verify & Save ---
print("Cluster 0 with Market Cap:")
print(df_cluster0_with_cap)


Cluster 0 with Market Cap:
           Date   Stock  PredictedReturn  PredictedVolatility  Cluster  \
0    2024-10-31     AAP        -0.022118         2.359802e-01        0   
1    2024-10-31  AAPL.O         6.741405         7.493543e-02        0   
2    2024-10-31     ABT         0.645390         3.400000e-07        0   
3    2024-10-31  ACAD.O         1.150560         3.220000e-14        0   
4    2024-10-31  ACCS.K        -0.203058         1.690312e-01        0   
..          ...     ...              ...                  ...      ...   
464  2024-10-31       X         0.089402         1.272337e-01        0   
465  2024-10-31     XOM         0.300670         5.043796e-02        0   
466  2024-10-31  XRAY.O         1.176372         8.663112e-02        0   
467  2024-10-31  YORW.O         0.344406         5.938221e-02        0   
468  2024-10-31  ZYXI.O         0.427867         1.423785e-01        0   

     Company Market Cap  
0          2.247617e+09  
1          3.210436e+12  
2     

In [241]:
# Rank and select top 10 by market cap for each cluster
def get_top10_by_market_cap(cluster_df):
    return (
        cluster_df.sort_values('Company Market Cap', ascending=False)
        .head(10)
        .reset_index(drop=True)
    )

# Apply to all clusters
top10_cluster0 = get_top10_by_market_cap(df_cluster0_with_cap)
top10_cluster1 = get_top10_by_market_cap(df_cluster1_with_cap)
top10_cluster2 = get_top10_by_market_cap(df_cluster2_with_cap)
top10_cluster3 = get_top10_by_market_cap(df_cluster3_with_cap)

# Display results
print("Top 10 Stocks in Cluster 0 by Market Cap:")
print(top10_cluster0[['Stock', 'Company Market Cap']])

print("\nTop 10 Stocks in Cluster 1 by Market Cap:")
print(top10_cluster1[['Stock', 'Company Market Cap']])

print("\nTop 10 Stocks in Cluster 2 by Market Cap:")
print(top10_cluster2[['Stock', 'Company Market Cap']])

print("\nTop 10 Stocks in Cluster 3 by Market Cap:")
print(top10_cluster3[['Stock', 'Company Market Cap']])


Top 10 Stocks in Cluster 0 by Market Cap:
    Stock  Company Market Cap
0  AAPL.O        3.210436e+12
1  NVDA.O        2.968748e+12
2  MSFT.O        2.888548e+12
3  AMZN.O        2.097821e+12
4     LLY        7.713173e+11
5     WMT        6.842381e+11
6     XOM        4.855501e+11
7      MA        4.810539e+11
8     UNH        4.469742e+11
9  ORCL.K        4.185880e+11

Top 10 Stocks in Cluster 1 by Market Cap:
    Stock  Company Market Cap
0     SYK        1.410049e+11
1   HON.O        1.361384e+11
2      KR        4.710676e+10
3     VLO        4.067620e+10
4  ODFL.O        3.453858e+10
5     PPG        2.575484e+10
6  TROW.O        2.071391e+10
7    ON.O        1.820118e+10
8     BBY        1.528425e+10
9  ENTG.O        1.486634e+10

Top 10 Stocks in Cluster 2 by Market Cap:
    Stock  Company Market Cap
0      HD        3.489782e+11
1     MRK        2.388872e+11
2  ADBE.O        1.716714e+11
3  AMGN.O        1.685266e+11
4     PFE        1.458698e+11
5     UNP        1.435361e+11
6 

In [242]:
# 1. Get the list of top 10 stocks from each cluster
top10_stocks_cluster0 = top10_cluster0['Stock'].unique().tolist()
top10_stocks_cluster1 = top10_cluster1['Stock'].unique().tolist()
top10_stocks_cluster2 = top10_cluster2['Stock'].unique().tolist()
top10_stocks_cluster3 = top10_cluster3['Stock'].unique().tolist()

# 2. Filter the original clustered_df for each cluster's top 10 stocks
alldate_cluster0 = clustered_df[clustered_df['Stock'].isin(top10_stocks_cluster0)].drop(columns=['Cluster'])
alldate_cluster1 = clustered_df[clustered_df['Stock'].isin(top10_stocks_cluster1)].drop(columns=['Cluster'])
alldate_cluster2 = clustered_df[clustered_df['Stock'].isin(top10_stocks_cluster2)].drop(columns=['Cluster'])
alldate_cluster3 = clustered_df[clustered_df['Stock'].isin(top10_stocks_cluster3)].drop(columns=['Cluster'])

# 3. Reset index for clean numbering
alldate_cluster0.reset_index(drop=True, inplace=True)
alldate_cluster1.reset_index(drop=True, inplace=True)
alldate_cluster2.reset_index(drop=True, inplace=True)
alldate_cluster3.reset_index(drop=True, inplace=True)

# 4. Verify the results
print(f"Cluster 0 (Top 10 stocks) size: {len(alldate_cluster0)}")
print(f"Cluster 1 (Top 10 stocks) size: {len(alldate_cluster1)}")
print(f"Cluster 2 (Top 10 stocks) size: {len(alldate_cluster2)}")
print(f"Cluster 3 (Top 10 stocks) size: {len(alldate_cluster3)}")

print("\nSample from Cluster 0:")
print(alldate_cluster0)

Cluster 0 (Top 10 stocks) size: 96
Cluster 1 (Top 10 stocks) size: 100
Cluster 2 (Top 10 stocks) size: 96
Cluster 3 (Top 10 stocks) size: 96

Sample from Cluster 0:
          Date   Stock  PredictedReturn  PredictedVolatility
0   2024-01-31  AAPL.O         6.281893         8.009232e-02
1   2024-01-31  AMZN.O         2.548663         5.700000e-11
2   2024-01-31     LLY         0.804685         6.553838e-02
3   2024-01-31      MA         3.381480         6.677547e-02
4   2024-01-31  MSFT.O         1.990512         6.507592e-01
..         ...     ...              ...                  ...
91  2024-10-31  NVDA.O         5.503010         1.234098e-01
92  2024-10-31  ORCL.K         2.110557         9.266789e-02
93  2024-10-31     UNH        -0.040389         5.174712e-02
94  2024-10-31     WMT        -0.171602         1.281258e-01
95  2024-10-31     XOM         0.300670         5.043796e-02

[96 rows x 4 columns]


In [243]:
import pandas as pd

# Function to pivot and replace NaN with 0
def pivot_and_fill(df, value_column):
    """Pivot the DataFrame and replace NaN with 0"""
    return df.pivot_table(
        index='Date',
        columns='Stock',
        values=value_column,
        aggfunc='first'
    ).fillna(0)  # This converts NaN to 0

# Process PredictedReturn (with NaN→0)
return_cluster0 = pivot_and_fill(alldate_cluster0[['Date', 'Stock', 'PredictedReturn']], 'PredictedReturn')
return_cluster1 = pivot_and_fill(alldate_cluster1[['Date', 'Stock', 'PredictedReturn']], 'PredictedReturn')
return_cluster2 = pivot_and_fill(alldate_cluster2[['Date', 'Stock', 'PredictedReturn']], 'PredictedReturn')
return_cluster3 = pivot_and_fill(alldate_cluster3[['Date', 'Stock', 'PredictedReturn']], 'PredictedReturn')

# Process PredictedVolatility (with NaN→0)
volatility_cluster0 = pivot_and_fill(alldate_cluster0[['Date', 'Stock', 'PredictedVolatility']], 'PredictedVolatility')
volatility_cluster1 = pivot_and_fill(alldate_cluster1[['Date', 'Stock', 'PredictedVolatility']], 'PredictedVolatility')
volatility_cluster2 = pivot_and_fill(alldate_cluster2[['Date', 'Stock', 'PredictedVolatility']], 'PredictedVolatility')
volatility_cluster3 = pivot_and_fill(alldate_cluster3[['Date', 'Stock', 'PredictedVolatility']], 'PredictedVolatility')

# Divide all values by 100 in return DataFrames
return_cluster0 = return_cluster0 / 100
return_cluster1 = return_cluster1 / 100
return_cluster2 = return_cluster2 / 100
return_cluster3 = return_cluster3 / 100

# Verify results
print("PredictedReturn with NaN→0 (Cluster 0):")
print(return_cluster0.head())

print("\nPredictedVolatility with NaN→0 (Cluster 0):")
print(volatility_cluster0.head())

PredictedReturn with NaN→0 (Cluster 0):
Stock         AAPL.O    AMZN.O       LLY        MA    MSFT.O    NVDA.O  \
Date                                                                     
2024-01-31  0.062819  0.025487  0.008047  0.033815  0.019905  0.056254   
2024-02-29  0.094539  0.027849 -0.010776  0.047911  0.018627  0.056854   
2024-03-31 -0.012371  0.029559  0.005656  0.028336  0.018647  0.049679   
2024-04-30  0.052672  0.035016  0.014877  0.028785  0.019831  0.068073   
2024-05-31  0.039875  0.026350 -0.004693  0.035818  0.000000  0.066200   

Stock         ORCL.K       UNH       WMT       XOM  
Date                                                
2024-01-31  0.035475  0.032165 -0.041065  0.013605  
2024-02-29  0.024010  0.010499  0.026396  0.011843  
2024-03-31  0.015289  0.012276 -0.028806  0.003799  
2024-04-30  0.039447  0.027219 -0.017328  0.018659  
2024-05-31  0.022564 -0.003671  0.013885  0.014666  

PredictedVolatility with NaN→0 (Cluster 0):
Stock         AAPL.O     

In [244]:
# Extract stock names (column headers) from each cluster
stocks_cluster0 = return_cluster0.columns.tolist()
stocks_cluster1 = return_cluster1.columns.tolist() 
stocks_cluster2 = return_cluster2.columns.tolist()
stocks_cluster3 = return_cluster3.columns.tolist()


optimal_weights = portfolio_optimization(return_cluster0, stocks_cluster0)
print("Allocation of = Cluster0")

optimal_weights = portfolio_optimization(return_cluster1, stocks_cluster1)
print("Allocation of = Cluster1")

optimal_weights = portfolio_optimization(return_cluster2, stocks_cluster2)
print("Allocation of = Cluster2")

optimal_weights = portfolio_optimization(return_cluster3, stocks_cluster3)
print("Allocation of = Cluster3")

Annualized Stock Return = Stock
AAPL.O    0.701630
AMZN.O    0.334417
LLY       0.072734
MA        0.496209
MSFT.O    0.171625
NVDA.O    1.008753
ORCL.K    0.384711
UNH       0.165712
WMT      -0.005373
XOM       0.172587
dtype: float64
Optimal Portfolio Allocation max SR
AAPL.O: 1.19%
AMZN.O: 16.18%
LLY: 0.00%
MA: 18.70%
MSFT.O: 12.10%
NVDA.O: 51.82%
ORCL.K: 0.00%
UNH: 0.00%
WMT: 0.00%
XOM: 0.00%

Expected Annual Return: 69.88%
Annual Volatility: 1.21%
Sharpe Ratio: 57.67
Allocation of = Cluster0
Annualized Stock Return = Stock
BBY      -0.073426
ENTG.O   -0.203435
HON.O     0.069405
KR        0.086160
ODFL.O   -0.211377
ON.O     -0.046246
PPG      -0.021807
SYK       0.271659
TROW.O    0.146765
VLO       0.150804
dtype: float64
Optimal Portfolio Allocation max SR
BBY: 0.00%
ENTG.O: 0.00%
HON.O: 0.00%
KR: 0.00%
ODFL.O: 0.00%
ON.O: 0.00%
PPG: 0.00%
SYK: 66.09%
TROW.O: 33.91%
VLO: 0.00%

Expected Annual Return: 22.93%
Annual Volatility: 11.58%
Sharpe Ratio: 1.98
Allocation of = Cluster1

In [245]:


# Read clustered data
monthly_df = pd.read_csv(r"Output/models/monthly_returns.csv")



print(monthly_df) 

print(return_cluster0)

           Date       GSG       IEF       SPY       VNQ
0    2006-08-01 -0.071330  0.017597  0.021823  0.034843
1    2006-09-01 -0.107363  0.012216  0.022504  0.010101
2    2006-10-01 -0.025434  0.005368  0.036053  0.070310
3    2006-11-01  0.050000  0.013220  0.019885  0.047812
4    2006-12-01 -0.069222 -0.015514  0.007757 -0.037379
..          ...       ...       ...       ...       ...
220  2024-12-01  0.029801 -0.025775 -0.027334 -0.092502
221  2025-01-01  0.034451  0.012589  0.030312  0.026337
222  2025-02-01 -0.014654  0.024726 -0.012695  0.036996
223  2025-03-01  0.026138  0.003719 -0.058551 -0.035783
224  2025-04-01  0.004611  0.002413  0.014340  0.015126

[225 rows x 5 columns]
Stock         AAPL.O    AMZN.O       LLY        MA    MSFT.O    NVDA.O  \
Date                                                                     
2024-01-31  0.062819  0.025487  0.008047  0.033815  0.019905  0.056254   
2024-02-29  0.094539  0.027849 -0.010776  0.047911  0.018627  0.056854   
2024-03-

In [248]:
print(return_cluster0)
print(return_cluster0.std())

Stock         AAPL.O    AMZN.O       LLY        MA    MSFT.O    NVDA.O  \
Date                                                                     
2024-01-31  0.062819  0.025487  0.008047  0.033815  0.019905  0.056254   
2024-02-29  0.094539  0.027849 -0.010776  0.047911  0.018627  0.056854   
2024-03-31 -0.012371  0.029559  0.005656  0.028336  0.018647  0.049679   
2024-04-30  0.052672  0.035016  0.014877  0.028785  0.019831  0.068073   
2024-05-31  0.039875  0.026350 -0.004693  0.035818  0.000000  0.066200   
2024-06-30  0.062127  0.023261  0.009206  0.038735  0.000000  0.059997   
2024-07-31 -0.028936  0.022828  0.023161  0.033809  0.019692  0.067084   
2024-08-31  0.040778  0.024575  0.007734  0.039150  0.000000  0.056645   
2024-09-30  0.080721  0.000000  0.000992  0.035583  0.018434  0.062826   
2024-10-31  0.067414  0.028783  0.004881  0.019783  0.018111  0.055030   

Stock         ORCL.K       UNH       WMT       XOM  
Date                                                
2024-